In [3]:
mask = 0x1
i = 0
comm_size=21
rank=10

while (mask < comm_size):
    dst = rank ^ mask
    
    dst_tree_root = dst >> i
    dst_tree_root <<= i
    
    my_tree_root = rank >> i
    my_tree_root <<= i
    if dst<comm_size:
        print(i, dst)
    
    if (dst_tree_root + mask > comm_size):
        nprocs_completed = comm_size - my_tree_root - mask
        print('nprocs_completed: ', nprocs_completed)
        j = mask
        k = 0
        while (j):
            j >>= 1
            k += 1
        k -= 1
        tmp_mask = mask >> 1
        
        while (tmp_mask):
            dst = rank ^ tmp_mask
            tree_root = rank >> k
            tree_root <<= k
            if ((dst > rank) and (rank < tree_root + nprocs_completed) and (dst >= tree_root + nprocs_completed)):
                print('{}- Send to {}'.format(i, dst))
            elif ((dst < rank) and (dst < tree_root + nprocs_completed) and (rank >= tree_root + nprocs_completed)):
                print('{}- Receive {} from {}'.format(i, comm_size - (my_tree_root + mask), dst))
            
            tmp_mask >>= 1
            k -= 1
    
    mask <<= 1
    i+=1

0 11
1 8
2 14
3 2
nprocs_completed:  5
4- Receive 5 from 2
4- Send to 14


In [7]:
n = 3
p = 3

size = n*p


# Inter-Node
for i in range(size):
    left_index = (size + i - p) % size
    right_index = (size + i + p) % size
    
    send_index = i
    recv_index = left_index
    
    for j in range(n-1):
        send_rank = send_index
        recv_rank = recv_index
        
        #print('{} is going to send from {} to {} and receive from {} at {}'.format(i, send_rank, right_index, left_index, recv_rank))
    
        send_index = recv_index
        recv_index = (size + recv_index - p) % size
        
        
# Intra_Node
nodes = [0 for x in range(size)]
for i in range(n):
    for j in range(p):
        nodes[i*p+j] = i
        

for i in range(size):
    right_index = i + 1
    if(right_index >= size):
        right_index -= p
    elif(nodes[i] != nodes[right_index]):
        right_index -= p
        
    left_index = i - 1
    if(left_index < 0):
        left_index += p
    elif(nodes[i] != nodes[right_index]):
        left_index += p
    
    send_index = i
    recv_index = left_index
    
    for j in range(1, p):
        for k in range(n):
            send_rank = (size + send_index - k*p) % size
            recv_rank = (size + recv_index - k*p) % size
            print('{} is going to send from {} to {} and receive from {} at {}'.format(i, send_rank, right_index, left_index, recv_rank))
        send_index = recv_index
        recv_index = recv_index - 1
        if recv_index < 0:
            recv_index += p
        elif(nodes[i] != nodes[recv_index]):
            recv_index += p
            

    

0 is going to send from 0 to 1 and receive from 2 at 2
0 is going to send from 6 to 1 and receive from 2 at 8
0 is going to send from 3 to 1 and receive from 2 at 5
0 is going to send from 2 to 1 and receive from 2 at 1
0 is going to send from 8 to 1 and receive from 2 at 7
0 is going to send from 5 to 1 and receive from 2 at 4
1 is going to send from 1 to 2 and receive from 0 at 0
1 is going to send from 7 to 2 and receive from 0 at 6
1 is going to send from 4 to 2 and receive from 0 at 3
1 is going to send from 0 to 2 and receive from 0 at 2
1 is going to send from 6 to 2 and receive from 0 at 8
1 is going to send from 3 to 2 and receive from 0 at 5
2 is going to send from 2 to 0 and receive from 1 at 1
2 is going to send from 8 to 0 and receive from 1 at 7
2 is going to send from 5 to 0 and receive from 1 at 4
2 is going to send from 1 to 0 and receive from 1 at 0
2 is going to send from 7 to 0 and receive from 1 at 6
2 is going to send from 4 to 0 and receive from 1 at 3
3 is going